In [14]:
from openmeteopy import OpenMeteo
from openmeteopy.hourly import HourlyForecast
from openmeteopy.daily import DailyForecast
from openmeteopy.options import ForecastOptions
from itertools import product
import pandas as pd
from folium.plugins import HeatMap
import folium
import os


# Data Query Prototyping

In [15]:
# Creating hourly forecast object
hourly = HourlyForecast()


latitude = 45.5152
longitude = -122.6784

# Defining variables we want to return
hourly = hourly.temperature_2m()
hourly = hourly.cloudcover()
hourly = hourly.windspeed_10m()

# Selecting options
options = ForecastOptions(latitude = latitude, 
                          longitude = longitude,
                          forecast_days=3)

# Pulling Data
client = OpenMeteo(options, hourly)

# Download data
meteo = client.get_pandas()
print(meteo)

                  temperature_2m  cloudcover  windspeed_10m
time                                                       
2024-10-31T00:00             9.2          98           10.1
2024-10-31T01:00             8.2         100           11.7
2024-10-31T02:00             7.7          99            5.5
2024-10-31T03:00             7.4         100            5.1
2024-10-31T04:00             7.5         100            5.5
...                          ...         ...            ...
2024-11-06T19:00             8.9          37            6.4
2024-11-06T20:00            10.4          26            7.1
2024-11-06T21:00            11.4          14            7.4
2024-11-06T22:00            11.7          13            7.3
2024-11-06T23:00            11.7          11            7.2

[168 rows x 3 columns]


In [16]:
def return_surrounding_weather(latitude, longitude, margin = 0.01):
    
    latitudes = [latitude, latitude + margin, latitude - margin]
    longitudes = [longitude, longitude + margin, longitude - margin]

    # all locations to query
    locations = list(product(latitudes, longitudes))
    output_data = []
    for location in locations:

        hourly = HourlyForecast()
        # Extracting latitude and longtitude for call
        lat_val = location[0]
        long_val = location[1]
        # Defining variables we want to return
        hourly = hourly.temperature_2m()
        hourly = hourly.cloudcover()
        hourly = hourly.windspeed_10m()

        # Selecting options
        options = ForecastOptions(latitude = lat_val, 
                                longitude = long_val,
                                forecast_days=3)

        # Pulling Data
        client = OpenMeteo(options, hourly)

        # Download data
        sample = client.get_pandas()
        sample['latitude'] = lat_val
        sample['longitude'] = long_val
        sample['location'] = str(location)
        #sample = sample.reset_index()
        output_data.append(sample)

    combined_weather_data = pd.concat(output_data)
    return combined_weather_data


In [17]:
latitude = 45.5152
longitude = -122.6784
combined_weather = return_surrounding_weather(latitude, longitude, margin = 0.02).reset_index()

In [18]:
combined_weather.to_csv('weather_data.csv')

# Geo Plot Prototyping

In [19]:
def plotPoint(row, map):

    '''Takes row in dataframe and folium map object and plots latitude/longitude
    point in dataframe onto map object'''

    folium.CircleMarker(location=[row.latitude, row.longitude],
                        radius=4,
                        weight=row.temperature_2m).add_to(map)


In [23]:
# Reading data
df1 = pd.read_csv("c://Users//seelc//OneDrive//Desktop//Lucas Desktop Items//Projects//forecasting//Data//weather_data.csv")

# Creating datafrom of last measurement from each location
number_of_locations = len(df1['location'].drop_duplicates())
df1 = df1.sort_values(by = 'time', ascending=True).reset_index(drop=True)
last_observation_df = df1.head(number_of_locations)
df1.head(10)

,Unnamed: 0,time,temperature_2m,cloudcover,windspeed_10m,latitude,longitude,location
0,0,2024-10-31T00:00,9.2,98,10.1,45.5152,-122.6784,"(45.5152, -122.6784)"
1,168,2024-10-31T00:00,9.4,98,7.6,45.5152,-122.6584,"(45.5152, -122.6584)"
2,1344,2024-10-31T00:00,7.5,100,14.0,45.4952,-122.6984,"(45.4952, -122.69839999999999)"
3,1008,2024-10-31T00:00,8.9,98,10.1,45.4952,-122.6784,"(45.4952, -122.6784)"
4,336,2024-10-31T00:00,8.9,100,9.7,45.5152,-122.6984,"(45.5152, -122.69839999999999)"
5,672,2024-10-31T00:00,9.2,98,7.4,45.5352,-122.6584,"(45.5352, -122.6584)"
6,840,2024-10-31T00:00,9.4,100,9.7,45.5352,-122.6984,"(45.5352, -122.69839999999999)"
7,1176,2024-10-31T00:00,9.3,98,7.6,45.4952,-122.6584,"(45.4952, -122.6584)"
8,504,2024-10-31T00:00,9.5,98,7.4,45.5352,-122.6784,"(45.5352, -122.6784)"
9,1009,2024-10-31T01:00,7.8,100,11.7,45.4952,-122.6784,"(45.4952, -122.6784)"


In [24]:
portland_coordinates = [45.523, -122.676]
map = folium.Map(location= portland_coordinates, 
                 zoom_start=10, 
                 control_scale=True)

# Iterating through dataframe and plotting points
for index, row in last_observation_df.iterrows():
    plotPoint(row, map)

map

### Creating Heatmap

In [27]:

# Creating base map
base_map = folium.Map(location=portland_coordinates,
                   control_scale=True, 
                   zoom_start=10)

# Extracting data
map_values1 = df1[['latitude', 'longitude', 'temperature_2m']]
data = map_values1.values.tolist()

# Creating heatmap
hm = HeatMap(data,gradient={0.1: 'blue', 0.3: 'lime', 0.5: 'yellow', 0.7: 'orange', 1: 'red'}, 
                min_opacity=0.05, 
                max_opacity=0.9, 
                radius=50,
                use_local_extrema=False)

# Adding to basemap
base_map.add_child(hm)

#m = folium.Map(data_for_heatmap.values,  tiles = 'stamentoner', zoom_start=10, control_scale=True)

### Creating Heatmap with Time Dimension

In [ ]:
m = folium.Map([48.0, 5.0], zoom_start=10)

# Creating time index's
time_index = df1['time'].drop_duplicates().to_list()

hm = folium.plugins.HeatMapWithTime(data, index=time_index, auto_play=True, max_opacity=0.3)

hm.add_to(m)